In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 11 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

02 January 2023 ; 18:24:22


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,14,0,7.9,1.4,2.6,0.514,...,1.75,112.78,110.93,1.85,1.69,112.84,111.02,1.82,1,1
11,MarJon Beauchamp,SF,22,MIL,25,8,15.0,2.2,5.6,0.403,...,1.75,112.78,110.93,1.85,1.69,112.84,111.02,1.82,1,1
17,A.J. Lawson,SG,22,DAL,2,0,1.5,0.5,1.0,0.500,...,2.24,116.75,114.46,2.29,2.31,116.70,114.35,2.35,1,1
25,Jaden Hardy,SG,20,DAL,11,0,7.1,1.4,4.0,0.341,...,2.24,116.75,114.46,2.29,2.31,116.70,114.35,2.35,1,1
36,AJ Griffin,SF,19,ATL,32,10,21.8,4.2,9.0,0.464,...,-0.97,113.22,114.22,-1.00,-1.34,113.33,114.68,-1.35,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,34.1,ORL,16.3,0.820000,3.280000,6.9,20.9,13,0.317771
1,Bennedict Mathurin,28.1,IND,13.5,-0.886486,2.216216,5.4,17.2,6,0.125938
2,Jalen Duren,24.2,DET,15.3,0.227778,4.555556,3.1,7.4,15,0.089349
3,Jaden Ivey,29.9,DET,12.1,-0.702857,0.000000,5.2,15.1,15,0.088043
4,Walker Kessler,18.8,UTA,19.8,1.366667,6.150000,2.9,6.9,10,0.081442


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)